<a href="https://colab.research.google.com/github/dqniellew1/DLPT/blob/master/Real_World_Data_representations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Images

`imageio` package to handle images

In [0]:
drive_dir = 'drive/My Drive/dlwpt-code/data/'

In [0]:
import torch
import imageio

In [30]:
# Loading an image
img_arr = imageio.imread(drive_dir + '/p1ch4/image-dog/bobby.jpg')
img_arr.shape

(720, 1280, 3)

PyTorch require for input tensors to be in the format of `C x H x W`.
We can use `permute` method to get the new format.

In [0]:
img = torch.from_numpy(img_arr)
out = img.permute(2, 0, 1)

To store images in a batch in tensor, the dimensions are a `N x C x H x W` 
tensor.

A more efficient alternative to using `stack` to build up the tensor, we can pre-allocate a tensor of approiprate size and fill it with images loaded from a directory.

In [0]:
batch_size = 100
batch = torch.zeros(100, 3, 256, 256, dtype=torch.uint8)

## Loading images from a directory

In [0]:
import os


data_dir = 'drive/My Drive/dlwpt-code/data/p1ch4/image-cats/'
filenames = [name for name in os.listdir(data_dir) if os.path.splitext(name)[-1] == '.png']
for i, filename in enumerate(filenames):
  img_arr = imageio.imread(os.path.join(data_dir, filename))
  img_t = torch.from_numpy(img_arr)
  img_t = img_t.permute(2, 0, 1)
  img_t = img_t[:3] # Keep only the first three channels. Only RGB inputs.
  batch[i] = img_t

Neural networks work best when the input data ranges from 0 to 1, or from -1 to 1. Typically we want to cast a tensor to floating point and normalize the values of the pixels.

Normalizations is trickier as we have to decide the range of input between (0 to 1) or (-1 to 1). One possibility is to just divide the values of pixels by 255. (The maximum nimber representable number in 8-bit unsigned)

In [0]:
batch = batch.float()
batch /= 255.0

Another way is to compute the **mean** and **std** of the input data and scale it so that the output has **zero mean** and **unit std** across each channel.

In [0]:
n_channels = batch.shape[1]
for c in range(n_channels):
  mean = torch.mean(batch[:, c])
  std = torch.std(batch[:, c])
  batch[:, c] = (batch[:, c] - mean) / std

Above is an example of normalizing a single image, because we do not know yet how to operate on an entire dataset. It is good practice to compute the mean and standard deviation on the entire training data in advance and then subtract and divide by these fixed pre-computed quantities.

# Volumetric Data

Consists of an added dimension after channel which is **depth**, leading to a 5D tensor of shape:

 `N x C x D x H x W`

In [32]:
# Loading in a sample CT scan
dir_path = drive_dir + "p1ch4/volumetric-dicom/2-LUNG 3.0  B70f-04083"
vol_arr = imageio.volread(dir_path, 'DICOM')
vol_arr.shape

Reading DICOM (examining files): 1/99 files (1.0%)2/99 files (2.0%)3/99 files (3.0%)4/99 files (4.0%)5/99 files (5.1%)6/99 files (6.1%)7/99 files (7.1%)8/99 files (8.1%)9/99 files (9.1%)10/99 files (10.1%)11/99 files (11.1%)12/99 files (12.1%)13/99 files (13.1%)14/99 files (14.1%)15/99 files (15.2%)16/99 files (16.2%)17/99 files (17.2%)18/99 files (18.2%)19/99 files (19.2%)20/99 files (20.2%)21/99 files (21.2%)22/99 files (22.2%)23/99 files (23.2%)24/99 files (24.2%)25/99 files (25.3%)26/99 files (26.3%)27/99 files (27

(99, 512, 512)

In this case, the layout is different from what PyTorch expects, due to having no channel information. We will have to make room for the `channel` dimension using `unsqueeze`.

In [36]:
vol = torch.from_numpy(vol_arr).float()
vol = torch.transpose(vol, 0, 2)
vol = torch.unsqueeze(vol, 0)

vol.shape

torch.Size([1, 512, 512, 99])

# Tabular Data

In [0]:
import csv
import numpy as np

In [38]:
wine_path = drive_dir + 'p1ch4/tabular-wine/winequality-white.csv'
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=';', skiprows=1)
wineq_numpy

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], dtype=float32)

In [42]:
col_list = next(csv.reader(open(wine_path), delimiter=';'))
wineq_numpy.shape, col_list

((4898, 12),
 ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol',
  'quality'])

Convert the NumPy array into a PyTorch tensor.

In [43]:
wineq = torch.from_numpy(wineq_numpy)

wineq.shape, wineq.dtype

(torch.Size([4898, 12]), torch.float32)

In [44]:
data = wineq[:, :-1]
data, data.shape

(tensor([[ 7.0000,  0.2700,  0.3600,  ...,  3.0000,  0.4500,  8.8000],
         [ 6.3000,  0.3000,  0.3400,  ...,  3.3000,  0.4900,  9.5000],
         [ 8.1000,  0.2800,  0.4000,  ...,  3.2600,  0.4400, 10.1000],
         ...,
         [ 6.5000,  0.2400,  0.1900,  ...,  2.9900,  0.4600,  9.4000],
         [ 5.5000,  0.2900,  0.3000,  ...,  3.3400,  0.3800, 12.8000],
         [ 6.0000,  0.2100,  0.3800,  ...,  3.2600,  0.3200, 11.8000]]),
 torch.Size([4898, 11]))

In [48]:
target = wineq[:,-1]
target, target.shape

(tensor([6., 6., 6.,  ..., 6., 7., 6.]), torch.Size([4898]))

In [50]:
# Treating labels as an integer vector of scores:
target = wineq[:, -1].long()
target

tensor([6, 6, 6,  ..., 6, 7, 6])

In PyTorch we can achieve `one-hot` encoding using the `scatter_` method, which fills the tensor with values from a source tensor along the indices provided as arguments.

The `scatter_` method reads plainly as for each row, take the index of the target label (which conincides with the score in our case) and use it as the column index to set the value to 1.0. The end-result is a tensor encoding categorical information.

In [56]:
target_onehot = torch.zeros(target.shape[0], 10)

target_onehot.scatter_(1, target.unsqueeze(1), 1.0)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

The `unsqueeze` method adds a singleton dimension, from a 1D tensor to a 2D tensor.

In [62]:
target_unsqueezed = target.unsqueeze(1)
target_unsqueezed

tensor([[6],
        [6],
        [6],
        ...,
        [6],
        [7],
        [6]])